<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/prueba_2_cabuxa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalación de dependencias necesarias
# transformers y accelerate se usan para cargar y ejecutar el modelo
# peft es necesaria para el adapter Cabuxa
!pip install -q transformers accelerate peft

In [2]:
# Imports básicos
# torch se usa para la inferencia con el modelo
# transformers carga el modelo base
# peft permite cargar el adapter Cabuxa
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

In [3]:
# Identificadores del modelo base y del adapter Cabuxa
BASE_MODEL_ID = "huggyllama/llama-7b"
ADAPTER_ID = "irlab-udc/cabuxa-7b"

# Tokenizer correcto
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_ID,
    use_fast=False
)

# Token de padding
tokenizer.pad_token = tokenizer.eos_token

# Modelo base
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16
)

# Aplicar adapter Cabuxa
model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_ID
)

model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [4]:
def generate_cabuxa_prompt(instruction: str, input_text: str | None = None) -> str:
    if input_text:
        return f"""Abaixo está unha instrución que describe unha tarefa, xunto cunha entrada que proporciona máis contexto.
Escribe unha resposta que responda adecuadamente a entrada.

### Instrución:
{instruction}

### Entrada:
{input_text}

### Resposta:"""
    else:
        return f"""Abaixo está unha instrución que describe unha tarefa.
Escribe unha resposta que responda adecuadamente a entrada.

### Instrución:
{instruction}

### Resposta:"""

def cabuxa_generate(prompt: str, max_new_tokens: int = 256) -> str:
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs.get("attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(model.device)

    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    gen_ids = out[0][input_ids.shape[-1]:]  # SOLO lo nuevo
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

def cabuxa_judge_concordancia(oracion: str) -> str:
    instruction = """
You are an LLM-as-a-judge that will evaluate a grammatical error correction model in Galician.

Your task is to evaluate the output of the grammatical error correction (GEC) model
and decide whether the correction is ADEQUATE with respect to number agreement
(singular/plural between determiner, noun, adjective, or verb).

Return the answer EXACTLY in the following format, with no additional text:

output_modelo: "<evaluated sentence>"
etiqueta: <0 or 1>
explicacion: "<brief and precise explanation in Galician>"

Criteria:
- etiqueta = 1 → the correction is NOT adequate (there is still a number agreement error)
- etiqueta = 0 → the correction is adequate (there is no number agreement error)
- The explanation must justify only the number agreement

YOU MUST NOT, UNDER ANY CIRCUMSTANCES, CORRECT THE MODEL OUTPUT YOU HAVE TO EVALUATE. YOU MUST LIMIT YOURSELF EXCLUSIVELY TO DECIDING WHETHER IT CONTAINS A NUMBER AGREEMENT ERROR OR NOT.
YOU MUST NOT EVALUATE OTHER TYPES OF ERRORS.
""".strip()

    prompt = generate_cabuxa_prompt(
        instruction=instruction,
        input_text=f'Oración:\n"{oracion}"'
    )

    return cabuxa_generate(prompt, max_new_tokens=256)

In [5]:
# Oraciones de ejemplo (sin modificar)
exemplos = [
    "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas.",
    "O grupos de estudantes que participou no proxecto presentou os resultados finais onte pola tarde.",
    "As propostas que chegaron desde os concellos máis pequenos foi analizadas polo equipo técnico.",
    "A maioría das persoas entrevistadas manifestou a súa opinión durante a sesión pública.",
    "Os relatorios anuais sobre o impacto ambiental foi revisados exhaustivamente pola comisión de avaliación.",
    "A valiosa colección de poemas inéditos do autor galego publicouse recentemente baixo un prestixioso selo independente.",
    "O paquete de medidas económicas recentemente aprobado é de aplicación inmediata en todos os sectores da economía.",
    "As adversas condicións climáticas dos últimos días obrigou a suspender completamente varias actividades programadas ao aire libre.",
    "A maioría dos participantes no curso de formación mostrou un grande interese en continuar con sesións prácticas adicionais.",
    "Os sabios consellos que me deches sobre a xestión do tempo foi moi útiles para poder tomar unha decisión acertada.",
]

# Ejecución del judge sobre cada oración
for e in exemplos:
    print(e)
    print("→", cabuxa_judge_concordancia(e))
    print()

As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas.
→ output_modelo: "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas."
etiqueta: 1
explicación: "Oración: As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas."

O grupos de estudantes que participou no proxecto presentou os resultados finais onte pola tarde.
→ output_modelo: "O grupos de estudantes que participou no proxecto presentou os resultados finais onte pola tarde."
etiqueta: 1
explicación: "Oración: O grupos de estudantes que participou no proxecto presentou os resultados finais onte pola tarde."

As propostas que chegaron desde os concellos máis pequenos foi analizadas polo equipo técnico.
→ output_modelo: "As propostas que chegaron desde os concellos máis pequenos foi analizadas polo equipo técnico."
etiqueta: 0
explicación: "Oración: As propostas que chegaron desde os concellos máis pequenos foi analizadas polo equi